In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'/Users/sidharthgoel/Desktop/deep_learning_disease_detection/Chickdis'

In [3]:
from pathlib import Path
from dataclasses import dataclass

In [7]:
@dataclass(frozen=True)
class BaseModelConfig():
    root_dir: Path
    base_model_path: Path
    updated_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [5]:
from Chickdis.constants import *
from Chickdis.utils.common import create_directories, read_yaml,get_size


In [9]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_base_model_config(self) -> BaseModelConfig:
        config = self.config.prepare_base_model
        params= self.params
        create_directories([config.root_dir])
        base_model_config = BaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_model_path=Path(config.base_model_updated_path),
            params_image_size= params.IMAGE_SIZE,
            params_learning_rate= params.LEARNING_RATE,
            params_include_top=params.INCLUDE_TOP,
            params_weights=params.WEIGHTS,
            params_classes=params.CLASSES
        )
        
        return base_model_config

In [11]:
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [16]:
class PrepareBaseModel:
    def __init__(self,config:BaseModelConfig):
        self.config=config
    
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
            model.save(path)
    
    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path,model=self.model)

    @staticmethod
    def _prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable=False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable=False
       # Add custom head
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(flatten_in)

        full_model=tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics=["accuracy"],
            loss=tf.keras.losses.CategoricalCrossentropy()
        )
        full_model.summary(
        )
        return full_model
    
    def update_base_model(self):
        self.full_model=self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_model_path,model=self.full_model)

        

In [17]:
try:
    config_manager=ConfigurationManager()
    prepare_model_config=config_manager.get_base_model_config()
    base_model=PrepareBaseModel(prepare_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    raise e

[2025-09-08 13:25:33,647: INFO: common]: Directory created at: artifacts]
[2025-09-08 13:25:33,649: INFO: common]: Directory created at: artifacts/prepare_base_model]
[2025-09-08 13:25:34,070: WARNING: saving_utils]: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2025-09-08 13:25:34,193: WARNING: optimizer]: At this time, the v2.11+ optimizer `tf.keras.optimizers.SGD` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.SGD`.]
[2025-09-08 13:25:34,194: WARNING: __init__]: There is a known slowdown when using v2.11+ Keras optimizers on M1/M2 Macs. Falling back to the legacy Keras optimizer, i.e., `tf.keras.optimizers.legacy.SGD`.]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 

/opt/homebrew/Caskroom/miniconda/base/envs/chicken/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
